# Project 3
Griffith Stites and Alex Hindelang

## Assumptions
* Ignoring temperature changes for the fluid and how this effects density. Just choosing based off of the ideal temperature from the engineeringtoolbox chart
* That the force of the penguin exerted on the fluid does not change based on fluid density.

## To Do
* Make penguin mass changable
* Make rho changeable
* Write plotting function. Complete with rho values and penguin mass

# Question

In [103]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

# Model

In [41]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
km = UNITS.kilometer
hr = UNITS.hour
s = UNITS.second
N = UNITS.newton
lb = UNITS.pound

<Unit('pound')>

rho_water

C_d = Coeffient of drag of the penguin

frontal_area = The frontal area of the penguin

velocity_init = starting velocity of the penguin

pool_length = length of the pool. An olympic swimming pool is 50 m

C_d and frontal_area are currently for the gentoo penguin

In [151]:
rho_water = 1000 * kg/m**3
rho_sea_water = 1025 * kg/m**3
rho_syrup = 1370 * kg/m**3
rho_propyl_alcohol = 800 * kg/m**3
rho_air = 1.2 * kg/m**3

gentoo_mass = 13 * lb
params = Params(rho = rho_sea_water,
                C_d = 0.07,
                frontal_area = 0.02 * m**2,
                velocity_init = 0 * m / s,
                pool_length = 50 * m,
                penguin_mass = gentoo_mass
)

,values
rho,1025.0 kilogram / meter ** 3
C_d,0.07
frontal_area,0.02 meter ** 2
velocity_init,0.0 meter / second
pool_length,50 meter
penguin_mass,13 pound


In [152]:
def make_system(params):
    """Make a system object.
    
    params: Params object with
    
    returns: System object"""
    unpack(params)
    init = State(x=0 * m, v=velocity_init)
    t_end = 100 * s
    
    return System(params, init=init, t_end=t_end)

In [153]:
system = make_system(params)

,values
rho,1025.0 kilogram / meter ** 3
C_d,0.07
frontal_area,0.02 meter ** 2
velocity_init,0.0 meter / second
pool_length,50 meter
penguin_mass,13 pound
init,x 0 meter v 0.0 meter / secon...
t_end,100 second


In [154]:
def force_drag(V, system):
    """Computes drag force in the opposite direction of 'v'.
    
    V: velocity
    system: System object with rho, C_d, area
    
    returns: scalar drag force
    """
    unpack(system)
    f_drag_mag = -rho * V**2 * C_d * frontal_area / 2
    return f_drag_mag

Force of the penguin can be determined by finding the maximum speed of a penguin in salt water. Assuming that the drag force of the salt water is equal to the force of the penguin swimming and by knowing the drag coefficient of the penguin we can determine the swimming force of the penguin.

In [135]:
def force_penguin(max_speed, system):
    """Computes the force the penguin exerts on the fluid.
    
    max_speed: max speed in the fluid
    system: System object with rho, C_d, area
    
    returns: scalar penguin force"""
    f_drag = force_drag(max_speed, system)
    f_penguin = - f_drag
    return f_penguin

Testing force_penguin


In [136]:
gentoo_max_speed = 36 * km / hr
gentoo_max_speed = 10 * m / s
force_penguin = force_penguin(gentoo_max_speed, system)

<Quantity(71.75000000000001, 'kilogram * meter / second ** 2')>

In [137]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State ()
    t: time
    system: System object with
    
    returns: derivatives of y and v
    """
    x, v = state
    unpack(system)    

    f_drag = force_drag(v, system)
    f_penguin = force_penguin
    
    f_total = f_penguin + f_drag
    
    dydt = v
    dvdt = f_total / penguin_mass #acceleration
    
    return dydt, dvdt

Testing the slope_func

In [138]:
dydt, dvdt = slope_func(init, 0, system)
print(dydt)
print(dvdt)

0.0 meter / second
5.51923076923077 kilogram * meter / pound / second ** 2


In [148]:
def event_func(state, t, system):
    """Return the ditance of the penguin in the pool.
    """
    x, v = state
    return x - system.pool_length

# Results

In [149]:
results, details = run_ode_solver(system, slope_func, events=event_func)
details

,values
sol,None
t_events,[[5.569349434087116]]
nfev,80
njev,0
nlu,0
status,1
message,A termination event occurred.
success,True


In [150]:
results

,x,v
0.000000,0.000000e+00,0.000000
0.000100,6.083910e-08,0.001217
0.001100,7.361530e-06,0.013385
0.011100,7.495758e-04,0.135055
0.111100,7.486737e-02,1.343670
0.580890,1.901737e+00,6.086846
1.114761,5.980395e+00,8.757309
1.679273,1.123676e+01,9.670216
2.269910,1.703982e+01,9.919746
3.010465,2.441846e+01,9.985304


Testing the Gentoo penguin in the different substances.

In [162]:
params.rho = rho_sea_water
system = make_system(params)
results, details = run_ode_solver(system, slope_func, events=event_func)

,values
rho,1.2 kilogram / meter ** 3
C_d,0.07
frontal_area,0.02 meter ** 2
velocity_init,0.0 meter / second
pool_length,50 meter
penguin_mass,13 pound
init,x 0 meter v 0.0 meter / secon...
t_end,100 second


The original question that inspired us to make this model: What happens if a penguin swims in syrup? (Ignoring the sugary effect on the penguin. I choose to believe that the penguin would just want to eat it right up).

In [163]:
params.rho = rho_syrup
system = make_system(params)
results, details = run_ode_solver(system, slope_func, events=event_func)

In [164]:
params.rho = rho_propyl_alcohol
system = make_system(params)
results, details = run_ode_solver(system, slope_func, events=event_func)

In [166]:
params.rho = rho_air
system = make_system(params)
results, details = run_ode_solver(system, slope_func, events=event_func)

Trying the sea water results with a different penguin.

In [161]:
params.penguin_mass = 

gentoo_max_speed = 36 * km / hr
gentoo_max_speed = 10 * m / s
#To do the penguin force calculation, the rho must be the rho of the material the data was gathered in
params.rho = rho_sea_water 
force_penguin = force_penguin(gentoo_max_speed, system)
system = make_system(params)

SyntaxError: invalid syntax (<ipython-input-161-22758934b8d9>, line 1)

# Interpretation